In [1]:
import os
import pandas as pd
import re
import pickle

#Preprocesamiento
import spacy
from spacy.lang.es.stop_words import STOP_WORDS #importar set de stopwords
from nltk.stem import SnowballStemmer #importar stemmer
nlp = spacy.load('es_core_news_sm') #python -m spacy download es

#Topic modeling
import gensim
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
from IPython.core.display import display, HTML
from gensim.models import HdpModel

#Elimina los warnings
import warnings
warnings.filterwarnings("ignore")

C:\Users\diego\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def tokenizer(doc, sep=None, vocabulary = None, homol_dict=None, lemmatization=False, stemming = False):
    '''
    Por defecto divide la sentencia por el carácter espacio.
    Ej: 'Data Mining is the best course'->['Data',  'Mining', 'is', 'the', 'best', 'course']
    
    Input: 
    1. doc: str, documento.
    2. sep: str, carácter para dividir el documento en tokens, por defecto es el espacio.
    3. vocabuary: set, si un vocabulario es dado filtra las palabras que no estan presentes en el.
    4. homol_dict: dict, diccionario de homologaciones.
    5. lemmatization: bool, si es True lleva las palabras a su lema.
    6. stemming: bool, si es True lleva las palabas a su raíz.
    
    Output: 
    list, lista de tokens.
    
    Nota: aplicar stemming y lemmatization al mismo tiempo no es correcto.
    '''
    doc = re.sub(r'\S+@\S+', '', doc) #elimina correos electrónicos  
    doc = re.sub(r'[\xa0]', '', doc) #elimina el patrón \xa0
    doc = re.sub(r'[^\w\s]','', doc) #elimina los símbolos de puntuación excepto underscore
    doc = re.sub(r'[_]', '', doc) #elimina underscore
    doc = re.sub(r'[a-zA-Z]+[0-9]+', '', doc) #elimina los tokens que contienen letras y números
    doc = re.sub(r'([ø ÿ þ])', ' ', doc) #reemplazas los símbolos contenidos por un espacio 
    doc = re.sub(r'[0-9]', '', doc) #elimina los tokens númericos
    tokens = doc.split(sep) #tokenización
    tokens = [word.lower() for word in tokens] #pasar todas las palabras a minúsculas
    
    
    if vocabulary is not None:
        #solo considera caracteres que tokens que estan en vocabulary
        tokens = [word for word in tokens if word in vocabulary]
    
    #if homol_dict is not None:
    
    if lemmatization==True:
        tokens = [nlp(word)[0].lemma_ for word in tokens]
        
    if stemming == True:
        stemmer = SnowballStemmer('spanish')
        tokens = [stemmer.stem(word) for word in tokens]
    
    return tokens


In [3]:
#Importar la base de datos
df = pd.read_pickle('../data/robos_prose_v1.pkl')
df.head()

#Importar vocabulario
with open('../data/vocabulary.pickle', 'rb') as f:
    vocabulary = pickle.load(f)

In [4]:
#Corpus tokenizado utilizando el vocabulario
data_tokenized = [tokenizer(doc, vocabulary=vocabulary) for doc in df['sin_relato']]

#Creamos el diccionario a partir de los textos procesados en el formato que necesita LDA en gensim
dictionary = Dictionary(data_tokenized)

#Transformamos el corpus al formato que requiere la librería
#El corpus contiene una representacion numerica de los textos, un texto es representado por una lista de tuplas
#donde el primer elemento de la tupla es la id de la palabra y el segundo es su frecuencia de aparición en el texto.
corpus = [dictionary.doc2bow(text) for text in data_tokenized]

#Guardamos el diccionario y el corpus
dictionary.save('../data/dictionary.dict')
MmCorpus.serialize('../data/corpus.mm', corpus)

In [21]:
lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=5, alpha='auto', eta='auto', random_state=0) 

In [22]:
#guardamos el modelo
lda_model.save('lda_model.model') 

In [23]:
#printear las topn palabras más probables de un tópico
lda_model.show_topic(topicid=1, topn=15)

[('mecanismo', 0.06720624),
 ('estacionado', 0.04111982),
 ('estacionamiento', 0.038681664),
 ('seguridad', 0.03460389),
 ('informados', 0.014400032),
 ('mall', 0.0138035305),
 ('casa', 0.012337404),
 ('encontraba', 0.010977054),
 ('vidrios', 0.01001468),
 ('ingresada', 0.008947119),
 ('ruido', 0.008045356),
 ('suelo', 0.007937963),
 ('ventana', 0.007869375),
 ('llaves', 0.007554966),
 ('vidrio', 0.00734365)]

In [49]:
lda_vis = pyLDAvis.gensim.prepare(topic_model=lda_model, corpus=corpus, dictionary=dictionary, sort_topics=True, R=30)
lda_vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.136171  0.148906       1        1  30.290878
0      0.317022 -0.006664       2        1  26.230618
4      0.109663  0.062348       3        1  17.705780
2     -0.103209 -0.270750       4        1  15.525748
1     -0.187305  0.066160       5        1  10.246981, topic_info=     Category          Freq             Term         Total  loglift  logprob
22    Default  22967.000000      estacionado  22967.000000  30.0000  30.0000
3953  Default   3236.000000        mecanismo   3236.000000  29.0000  29.0000
40    Default   8579.000000        domicilio   8579.000000  28.0000  28.0000
170   Default   7584.000000           llaves   7584.000000  27.0000  27.0000
21    Default  10841.000000            calle  10841.000000  26.0000  26.0000
81    Default   6274.000000  estacionamiento   6274.000000  25.0000  25.0000
52    Default   3851.000000        seguridad   3851.000000  24.0000  24.0000
325   Default   2012.000000           porton   2012.000000  23.0000  23.0000
474   Default   1818.000000          usuario   1818.000000  22.0000  22.0000
409   Default   2431.000000            bajan   2431.000000  21.0000  21.0000
41    Default   8775.000000       encontraba   8775.000000  20.0000  20.0000
300   Default   2203.000000             arma   2203.000000  19.0000  19.0000
625   Default   2053.000000            fuego   2053.000000  18.0000  18.0000
33    Default   7451.000000             casa   7451.000000  17.0000  17.0000
429   Default   2957.000000         personas   2957.000000  16.0000  16.0000
699   Default   2265.000000            tipos   2265.000000  15.0000  15.0000
582   Default   1844.000000            armas   1844.000000  14.0000  14.0000
276   Default   1774.000000          pistola   1774.000000  13.0000  13.0000
814   Default   1658.000000          bajaron   1658.000000  12.0000  12.0000
121   Default   1186.000000            chapa   1186.000000  11.0000  11.0000
51    Default   2940.000000           puerta   2940.000000  10.0000  10.0000
2747  Default   1324.000000          externo   1324.000000   9.0000   9.0000
308   Default   1472.000000            bajar   1472.000000   8.0000   8.0000
5     Default   1381.000000          empresa   1381.000000   7.0000   7.0000
9     Default   2428.000000       individuos   2428.000000   6.0000   6.0000
320   Default   1777.000000           vidrio   1777.000000   5.0000   5.0000
423   Default    856.000000       informados    856.000000   4.0000   4.0000
152   Default   1946.000000          sujetos   1946.000000   3.0000   3.0000
221   Default    727.000000             mall    727.000000   2.0000   2.0000
322   Default   1017.000000            abrir   1017.000000   1.0000   1.0000
...       ...           ...              ...           ...      ...      ...
127    Topic5    132.371765      información    133.925369   2.2665  -5.8965
221    Topic5    664.633972             mall    727.112244   2.1883  -4.2828
71     Topic5     99.559929         portugal    100.756348   2.2662  -6.1813
3001   Topic5     89.629105      mencionadas     90.454254   2.2690  -6.2864
1436   Topic5     85.553894       referencia     86.302338   2.2695  -6.3329
3460   Topic5    140.694153          levanto    146.308167   2.2391  -5.8355
222    Topic5    348.410522            plaza    394.518036   2.1539  -4.9287
423    Topic5    693.355286       informados    856.783447   2.0665  -4.2405
670    Topic5    118.290977            montt    122.709740   2.2415  -6.0089
128    Topic5    238.466568       recuperado    273.000641   2.1429  -5.3078
1045   Topic5    191.084427           correo    212.665283   2.1712  -5.5294
1033   Topic5    301.082092           marcha    396.687042   2.0024  -5.0747
52     Topic5   1666.162109        seguridad   3851.259766   1.4403  -3.3638
81     Topic5   1862.505127  estacionamiento   6274.189453   1.0637  -3.2524
772    Topic5    378.906952          

HdpModel(
    corpus,
    id2word,
    max_chunks=None,
    max_time=None,
    chunksize=256,
    kappa=1.0,
    tau=64.0,
    K=15,
    T=150,
    alpha=1,
    gamma=1,
    eta=0.01,
    scale=1.0,
    var_converge=0.0001,
    outputdir=None,
    random_state=None,
)

In [9]:
corpus1 = corpus[0:10000]
corpus2 = corpus[10000:20000]

In [44]:
hdp_model = HdpModel(T=20, K=5, corpus=corpus1, id2word=dictionary, random_state=0) 

In [45]:
hdp_model.print_topic(topicno=0, topn=30)

'0.004*"estacionado" + 0.003*"domicilio" + 0.002*"encontraba" + 0.002*"escapo" + 0.002*"sirena" + 0.002*"madrugada" + 0.002*"cortado" + 0.001*"dctos" + 0.001*"lmoquillaza" + 0.001*"cherokee" + 0.001*"izquierdo" + 0.001*"colorado" + 0.001*"inesperadamente" + 0.001*"continuacion" + 0.001*"buin" + 0.001*"registran" + 0.001*"parti" + 0.001*"policias" + 0.001*"minera" + 0.001*"vespucio" + 0.001*"principio" + 0.001*"movistar" + 0.001*"rompio" + 0.001*"gral" + 0.001*"calle" + 0.001*"villa" + 0.001*"toman" + 0.001*"frontier" + 0.001*"responde" + 0.001*"maule"'

In [47]:
hdp_model.update(corpus2)

In [52]:
hdp_model.update?

In [51]:
hdp_model.print_topic(topicno=19, topn=30)

'0.004*"estacionado" + 0.002*"violenta" + 0.002*"compaia" + 0.002*"diego" + 0.002*"colisionan" + 0.002*"identificar" + 0.001*"notificado" + 0.001*"manejar" + 0.001*"derech" + 0.001*"seminario" + 0.001*"puesta" + 0.001*"corredora" + 0.001*"intimidan" + 0.001*"electrónico" + 0.001*"ocasionando" + 0.001*"cilindro" + 0.001*"habitualmente" + 0.001*"desciende" + 0.001*"accion" + 0.001*"detener" + 0.001*"depues" + 0.001*"viajando" + 0.001*"usando" + 0.001*"portería" + 0.001*"mueve" + 0.001*"sacada" + 0.001*"sentada" + 0.001*"enviaron" + 0.001*"sacaba" + 0.001*"copiloto"'

In [15]:
hdp_model

In [ ]:
hdp_model = HdpModel(T=20, K=5, corpus=corpus2, id2word=dictionary, random_state=0) 

In [41]:
#guardamos el modelo
hdp_model.save('hdp_model.model') 

In [43]:
#printear las topn palabras más probables de un tópico
hdp_model.print_topics(num_topics=1, num_words=10)

[(0,
  '0.021*estacionado + 0.010*domicilio + 0.010*calle + 0.008*encontraba + 0.007*casa + 0.006*llaves + 0.005*puerta + 0.004*estacionamiento + 0.004*chapa + 0.003*personas')]

In [46]:
#comparar a través de métricas con lda

Probar pasandole data de forma incremental y ver la cantidad de tópicos que detecta.